#Data 08: Wine

*  포르투갈 비노 베르데(Vinho Verde) 지역의 레드와 화이트 와인의 데이터가 들어있는 데이터 셋이다.  
1980년대까지는 주로 스파클링 레드 와인이 생산되다가,  이후 상황이 역전되어 현재는 모든 비노 베르데의 85% 정도가 화이트 와인이다.
*   Data from: [Cortez et al., 2009](https://archive.ics.uci.edu/ml/datasets/wine+quality)

##1.데이터 둘러보기

In [ ]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#런타임 다시 시작

In [ ]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

plt.style.use('seaborn')
sns.set(font_scale=1)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
#드라이브에 올려준 csv 데이터 파일 불러오기
red = pd.read_csv('/content/drive/MyDrive/Python/08_Wine/winequality-red.csv', sep=';')
white = pd.read_csv('/content/drive/MyDrive/Python/08_Wine/winequality-white.csv', sep=';') 
#콤마가 아닌 경우 구분자를 넣어줘야 한다!

In [ ]:
#불러온 데이터 일단 보기
red.head(3)

<컬럼 설명>   
* fixed acidity: 결합산
* volatile acidity: 휘발성 산
* citric acid: 시트르산
* redisual sugar: 잔당 
* chlorides: 염화물
* free sulfur dioxide: 유리 이산화황
* total sulfur dioxide: 이산화황
* density: 비중  
* pH: 산도
* sulphates: 황 
* alcohol: 알콜 도수
* quality: 품질(0 ~ 10 사이의 값)

In [ ]:
red.info()

In [ ]:
white.info()

##2.데이터를 보고 질문 만들기

* 와인의 어떤 특성이 와인 퀄리티와 가장 밀접한 관련이 있을까? 
* 와인 퀄리티를 예측하는 모델을 만들어보자.

##3.데이터 정비하기

###3-1. 데이터 합치기

In [ ]:
#데이터를 합치기 전에 구분자 컬럼 넣어주기
red['tag'] = 'r'
red

In [ ]:
#데이터를 합치기 전에 구분자 컬럼 넣어주기
white['tag'] = 'w'
white

In [ ]:
#두개의 와인 데이터셋을 위아래로 합치기
wine = pd.concat([red, white])
wine

In [ ]:
wine.shape

In [ ]:
#결측치를 간단하게 시각화 해주는 패키지 
import missingno
missingno.matrix(wine, figsize =(14,7))

##4.EDA & Visualization

###4-1. 퀄리티 체크

In [ ]:
#합친 Wine set에 대해서 각 퀄리티 점수 당 몇 개의 와인이 있는지 확인해보자
plt.figure(figsize=(12,8))
sns.countplot(data = wine ,x='quality')

plt.title("Quality Counts", fontsize=15)
plt.xlabel("Quality", fontsize=14)
plt.ylabel("Counts", fontsize=14)

plt.grid()

In [ ]:
#red/white를 나눠서 그려보자
tag1 = wine.groupby(['tag'])['quality'].value_counts().unstack(0)
tag1

In [ ]:
#red/white 와인의 quality 분포는?
ax = tag1.plot(kind='bar', color=['red', 'lightyellow'] ,figsize = (12,6))

ax.set_title('Quality', size=20)

#막대 그래프 위에 값 주기
for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds #해당 막대그래프의 정보: 왼쪽, 아래, 막대그래프의 폭, 높이 정보
    ax.annotate("%.0f"%(height), (left+width/2, height*1.01), ha='center')

###4-2. 각 컬럼 데이터의 분포 확인

In [ ]:
#plot을 한번에 하나씩 그릴수도 있지만... 같은 plot을 그린다면 여러개를 한번에 그리자
#퀄리티에 따라서 각 컬럼의 값이 어떻게 변하는가?

#bot plot
for i in wine.columns:
    if i =="quality":
        break
    sns.boxplot('quality',i,data=wine)
    plt.show()

#음의 상관관계: volatile acidity, chlorides
#양의 상관관계: citric acid, alcohol

In [ ]:
#plot을 한번에 하나씩 그릴수도 있지만... 같은 plot을 그린다면 여러개를 한번에 그리자
#퀄리티에 따라서 각 컬럼의 값이 어떻게 변하는가?

#bar plot
for i in wine.columns:
    if i =="quality":
        break
    sns.barplot('quality',i,data=wine)
    plt.show()

#음의 상관관계: volatile acidity, chlorides
#양의 상관관계: citric acid, alcohol

###4-3. 상관관계 확인하기

In [ ]:
#연속형 변수의 분포 확인하기
f, ax = plt.subplots(6, 2, figsize=(15, 8))

sns.kdeplot(data=wine, x='fixed acidity', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[0,0])
sns.kdeplot(data=wine, x='volatile acidity', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[0,1])
sns.kdeplot(data=wine, x='citric acid', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[1,0])
sns.kdeplot(data=wine, x='residual sugar', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[1,1])
sns.kdeplot(data=wine, x='chlorides', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[2,0])
sns.kdeplot(data=wine, x='free sulfur dioxide', fill=True, common_norm=False, palette='crest',alpha=.5, hue='tag', ax=ax[2,1])
sns.kdeplot(data=wine, x='total sulfur dioxide', fill=True, common_norm=False, palette='crest',alpha=.5, hue='tag', ax=ax[3,0])
sns.kdeplot(data=wine, x='density', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[3,1])
sns.kdeplot(data=wine, x='pH', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[4,0])
sns.kdeplot(data=wine, x='sulphates', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[4,1])
sns.kdeplot(data=wine, x='alcohol', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[5,0])
sns.kdeplot(data=wine, x='quality', hue='tag', fill=True, common_norm=False, palette='crest',alpha=.5, ax=ax[5,1])

plt.show()

In [ ]:
#컬럼 간 상관관계를 heatmap으로 확인하자
f, ax = plt.subplots(1, 1, figsize=(9, 8))

corrmat = wine.corr()
sns.heatmap(corrmat, annot=True, cmap='RdYlBu_r', ax=ax)
plt.title('Correlation map for Wine Data', size=15)

##5.와인 퀄리티 예측하기 

###5-1. 모델링하기 전 전처리하기

In [ ]:
#와인 퀄리티 단순화 하기
# 3 ~ 7점 까지는 나쁜 와인 = 0, 8 이상은 좋은 와인 = 1
reviews = []
for score in wine['quality']:
    if score >= 3 and score < 7:
        reviews.append(0)
    else:
        reviews.append(1)

wine['good'] = reviews

In [ ]:
wine.head()

In [ ]:
wine.groupby(['tag', 'good'])['good'].agg(['count'])

In [ ]:
#레드와인과 화이트와인을 나눠서 모델링한 후 비교해보자
rw = wine[wine.tag=='r']
ww = wine[wine.tag=='w']

In [ ]:
#패키지 불러오기
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [ ]:
#good 컬럼이 타겟변수, 나머지 컬럼이 설명변수가 된다
#total
Xt = wine.iloc[:,:-3]
yt = wine['good']

#red
Xr = rw.iloc[:,:-3]
yr = rw['good']

#white
Xw = ww.iloc[:,:-3]
yw = ww['good']

In [ ]:
#Train 데이터와 Test 데이터 나누기
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size = 0.2, random_state = 42) #total
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size = 0.2, random_state = 42) #red
Xw_train, Xw_test, yw_train, yw_test = train_test_split(Xw, yw, test_size = 0.2, random_state = 42) #white

In [ ]:
#각 컬럼의 값들이 정규분포를 가지도록 scaling 해 주는 과정
scaler = StandardScaler()

#total
Xt_train = scaler.fit_transform(Xt_train)
Xt_test = scaler.fit_transform(Xt_test)

#red
Xr_train = scaler.fit_transform(Xr_train)
Xr_test = scaler.fit_transform(Xr_test)

#white
Xw_train = scaler.fit_transform(Xw_train)
Xw_test = scaler.fit_transform(Xw_test)

###5-2. Logistic Regression

In [ ]:
#total
logt = LogisticRegression()
logt.fit(Xt_train, yt_train)
pred_logt = logt.predict(Xt_test)

#red
logr = LogisticRegression()
logr.fit(Xr_train, yr_train)
pred_logr = logt.predict(Xr_test)

#white
logw = LogisticRegression()
logw.fit(Xw_train, yw_train)
pred_logw = logt.predict(Xw_test)

In [ ]:
print("Accuracy Score: ")
print("Total: {:.2f}, Red: {:.2f}, White: {:.2f}".format(accuracy_score(pred_logt,yt_test), accuracy_score(pred_logr,yr_test), accuracy_score(pred_logw,yw_test)))

#print("classification Report:\n",classification_report(pred_log,y_test))
#print("confusion Matrix:\n",confusion_matrix(pred_log,y_test))

###5-3. Support Vector Machine(SVM)

In [ ]:
#total
svct = SVC()
svct.fit(Xt_train, yt_train)
pred_svct = svct.predict(Xt_test)

#red
svcr = SVC()
svcr.fit(Xr_train, yr_train)
pred_svcr = svcr.predict(Xr_test)

#white
svcw = SVC()
svcw.fit(Xw_train, yw_train)
pred_svcw = svcw.predict(Xw_test)

In [ ]:
#결과 출력하기
print("Accuracy Score: ")
print("Total: {:.2f}, Red: {:.2f}, White: {:.2f}".format(accuracy_score(pred_svct,yt_test), accuracy_score(pred_svcr,yr_test), accuracy_score(pred_svcw,yw_test)))

#print("classification Report:\n",classification_report(pred_svc,y_test))
#print("confusion Matrix:\n",confusion_matrix(pred_svc,y_test))

###5-4. Random Forest Classifier 

In [ ]:
#total
rfct = RandomForestClassifier()
rfct.fit(Xt_train, yt_train)
pred_rfct = rfct.predict(Xt_test)

#red
rfcr = RandomForestClassifier()
rfcr.fit(Xr_train, yr_train)
pred_rfcr = rfcr.predict(Xr_test)

#white
rfcw = RandomForestClassifier()
rfcw.fit(Xw_train, yw_train)
pred_rfcw = rfcw.predict(Xw_test)

In [ ]:
#결과 출력하기
print("Accuracy Score: ")
print("Total: {:.2f}, Red: {:.2f}, White: {:.2f}".format(accuracy_score(pred_rfct,yt_test), accuracy_score(pred_rfcr,yr_test), accuracy_score(pred_rfcw,yw_test)))

#print("classification Report:\n",classification_report(pred_rfc,y_test))
#print("confusion Matrix:\n",confusion_matrix(pred_rfc,y_test))

###5-5. 모델링 결과 그래프로 나타내기

In [ ]:
#total, red, white의 결과를 한데 모으자
rest = [accuracy_score(pred_logt,yt_test), accuracy_score(pred_svct,yt_test), accuracy_score(pred_rfct,yt_test)]
resr = [accuracy_score(pred_logr,yr_test), accuracy_score(pred_svcr,yr_test), accuracy_score(pred_rfcr,yr_test)]
resw = [accuracy_score(pred_logw,yw_test), accuracy_score(pred_svcw,yw_test), accuracy_score(pred_rfcw,yw_test)]

In [ ]:
df = pd.DataFrame(np.c_[rest,resr,resw], columns=['Total', 'Red', 'White'], index=['Log', 'SVC', 'RFC'])
df

In [ ]:
ax = df.plot(kind='bar', color = ['green', 'red', 'lightyellow'], edgecolor='lightgray' ,figsize = (12,6))
ax.set_title('Wine Modeling Result', fontsize=15)
ax.set_xticklabels(['Log', 'SVC', 'RFC'], rotation=0)
ax.legend(title='Type', bbox_to_anchor= (1.03, 1))
plt.xlabel("Wine Set", fontsize=14)
plt.ylabel("Accuracy", fontsize=14)

#막대 그래프 위에 값 주기
for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds #해당 막대그래프의 정보: 왼쪽, 아래, 막대그래프의 폭, 높이 정보
    ax.annotate("%.2f"%(height), (left+width/2, height*1.01), ha='center')

##6.Review

* 와인의 어떤 특성이 와인 퀄리티와 가장 밀접한 관련이 있을까? 
> Correlation 분석을 통해 확인
* 와인 퀄리티를 예측하는 모델을 만들어보자.
> 레드/화이트 와인을 나누어서 분석해본 결과와 합친 결과를 비교해보았다.
모델별로 분석 셋에 대한 결과가 달랐으나, 종합적으로 RFC를 사용했을 때 예측 성능이 가장 뛰어났다.